In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [2]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
import LinearAlgebra: ldiv!, dot
using Test
using ForwardDiff
using SpecialFunctions
using BenchmarkTools
using TransportMap
using QuadGK
using AdaptiveTransportMap: derivative, vander, transform!, evaluate, ncoeff, optimize
using DiffResults
using Distributions
using Random
using LoopVectorization
using Optim
using NLsolve
using Polynomials
using Optim: InverseDiagonal
using MLDataUtils
using Test

### Expanded function

In [231]:
Nx = 3
Ne = 8
X = randn(Nx, Ne)'

8×3 Adjoint{Float64,Array{Float64,2}}:
 -0.899679   0.149212  -0.203239
 -1.20342   -0.196523  -0.484925
 -1.03261   -1.04501    0.367875
 -0.343662   0.424117   0.720608
  0.32033    0.345181   0.739602
  1.0377    -1.029     -0.610308
  0.36607    2.42988   -0.0927185
 -1.15715    0.803241   1.71867

In [269]:
Nx = 3
Ne = 8

X = Matrix([-0.899679   0.149212  -0.203239;
     -1.20342   -0.196523  -0.484925;
     -1.03261   -1.04501    0.367875;
     -0.343662   0.424117   0.720608;
      0.32033    0.345181   0.739602;
      1.0377    -1.029     -0.610308;
      0.36607    2.42988   -0.0927185;
     -1.15715    0.803241   1.71867]')

# X = randn(Nx, Ne)

B = MultiBasis(CstProHermite(3; scaled =true), Nx)

idx = [0 0 0; 0 0 1; 0 1 0; 0 1 1; 0 1 2; 1 0 0]
Nψ = size(idx,1)
coeff =  randn(Nψ)

f = ExpandedFunction(B, idx, coeff)

out = zeros(Ne, Nψ);

In [270]:
@btime evaluate_basis!($out, $f, $X)

  354.810 ns (13 allocations: 1.14 KiB)


8×6 Array{Float64,2}:
 1.0  0.62513   0.628113  0.392652   -0.0798022  0.515909
 1.0  0.595558  0.62555   0.372551   -0.180659   0.439761
 1.0  0.610607  0.480715  0.293528    0.107982   0.483821
 1.0  0.554722  0.603845  0.334966    0.241379   0.613242
 1.0  0.550889  0.613082  0.33774     0.249793   0.615622
 1.0  0.575458  0.484722  0.278937   -0.170238   0.482548
 1.0  0.630263  0.144345  0.0909753  -0.0084351  0.610809
 1.0  0.30182   0.537531  0.162237    0.278833   0.451934

In [260]:
evaluate_basis!(out, f, X)

8×6 Array{Float64,2}:
 1.0  0.62513   0.628113  0.392652   -0.0798022  0.515909
 1.0  0.595558  0.62555   0.372551   -0.180659   0.439761
 1.0  0.610607  0.480715  0.293528    0.107982   0.483821
 1.0  0.554722  0.603845  0.334966    0.241379   0.613242
 1.0  0.550889  0.613082  0.33774     0.249793   0.615622
 1.0  0.575458  0.484722  0.278937   -0.170238   0.482548
 1.0  0.630263  0.144345  0.0909753  -0.0084351  0.610809
 1.0  0.30182   0.537531  0.162237    0.278833   0.451934

In [261]:
@btime evaluate_basis!($out, $f, $X)

  357.426 ns (13 allocations: 1.14 KiB)


8×6 Array{Float64,2}:
 1.0  0.62513   0.628113  0.392652   -0.0798022  0.515909
 1.0  0.595558  0.62555   0.372551   -0.180659   0.439761
 1.0  0.610607  0.480715  0.293528    0.107982   0.483821
 1.0  0.554722  0.603845  0.334966    0.241379   0.613242
 1.0  0.550889  0.613082  0.33774     0.249793   0.615622
 1.0  0.575458  0.484722  0.278937   -0.170238   0.482548
 1.0  0.630263  0.144345  0.0909753  -0.0084351  0.610809
 1.0  0.30182   0.537531  0.162237    0.278833   0.451934

In [255]:
@btime evaluate_basis!($out, $f, $X)

  443.899 ns (12 allocations: 1.14 KiB)


8×6 Array{Float64,2}:
 1.0  0.62513   0.628113  0.392652   -0.0798022  0.515909
 1.0  0.595558  0.62555   0.372551   -0.180659   0.439761
 1.0  0.610607  0.480715  0.293528    0.107982   0.483821
 1.0  0.554722  0.603845  0.334966    0.241379   0.613242
 1.0  0.550889  0.613082  0.33774     0.249793   0.615622
 1.0  0.575458  0.484722  0.278937   -0.170238   0.482548
 1.0  0.630263  0.144345  0.0909753  -0.0084351  0.610809
 1.0  0.30182   0.537531  0.162237    0.278833   0.451934

In [250]:
@btime evaluate_basis!($out, $f, $X)

  428.472 ns (12 allocations: 1.14 KiB)


8×6 Array{Float64,2}:
 1.0  0.62513   0.628113  0.392652   -0.0798022  0.515909
 1.0  0.595558  0.62555   0.372551   -0.180659   0.439761
 1.0  0.610607  0.480715  0.293528    0.107982   0.483821
 1.0  0.554722  0.603845  0.334966    0.241379   0.613242
 1.0  0.550889  0.613082  0.33774     0.249793   0.615622
 1.0  0.575458  0.484722  0.278937   -0.170238   0.482548
 1.0  0.630263  0.144345  0.0909753  -0.0084351  0.610809
 1.0  0.30182   0.537531  0.162237    0.278833   0.451934

In [239]:
@btime evaluate_basis!($out, $f, $X)

  718.075 ns (30 allocations: 2.27 KiB)


8×6 Array{Float64,2}:
 1.0  0.62513   0.628113  0.392652   -0.0798022  0.515909
 1.0  0.595558  0.62555   0.372551   -0.180659   0.439761
 1.0  0.610607  0.480715  0.293528    0.107982   0.483821
 1.0  0.554722  0.603845  0.334966    0.241379   0.613242
 1.0  0.550889  0.613082  0.33774     0.249793   0.615622
 1.0  0.575458  0.484722  0.278937   -0.170238   0.482548
 1.0  0.630263  0.144345  0.0909753  -0.0084351  0.610809
 1.0  0.30182   0.537531  0.162237    0.278833   0.451934

### HermiteMap

In [5]:
Nx = 20
Ne = 300
m = 10
X = randn(Nx, Ne) .* randn(Nx, Ne) .+ rand(Nx);
X0 = deepcopy(X);

In [6]:
M = HermiteMap(m, X);

In [17]:
out = zero(X);
@time optimize(M, X, 6; apply_rescaling = true)

  0.615993 seconds (802.12 k allocations: 1.099 GiB, 9.78% gc time)


HermiteMap{10,20}(LinearTransform{20}([0.6452092755517298, 0.29687711276696244, 0.39792962261682446, 0.824283500094227, 0.6267945193911748, 0.6195809567042947, 0.7044328792803208, 0.19118372451754287, 0.7871523156004712, 0.8123027298275515, 0.1510945003928109, 0.10678934898743966, 0.826491259982191, 0.8039182921154494, 0.7814270666852019, 0.06773523022746741, 0.5233071202573701, 0.1842018773240075, 0.9889928883178202, -0.0020739482347915724], [0.9592727274191106 0.0 … 0.0 0.0; 0.0 1.0325551270246631 … 0.0 0.0; … ; 0.0 0.0 … 0.9094289075684655 0.0; 0.0 0.0 … 0.0 1.0193808884831794], true), HermiteMapk[HermiteMapk{10,6,1}(IntegratedFunction{10,6,1}(Rectifier("softplus"), ParametricFunction{10,6,1}(ExpandedFunction{10,6,1}(MultiBasis{10,1}(Basis of 10 functions:
ProPolyHermite{0}(ImmutablePolynomial(1.0), false)
ProHermite{0}(ProPolyHermite{0}(ImmutablePolynomial(0.6316187777460647), true), true)
ProHermite{1}(ProPolyHermite{1}(ImmutablePolynomial(0.6316187777460647*x), true), true)
ProHe

In [20]:
function timing()
    @btime begin
        Nx = 10
        Ne = 300
        m = 10
        X = randn(Nx, Ne) .* randn(Nx, Ne) .+ rand(Nx);
        X0 = deepcopy(X);   
#         M = HermiteMap(m, X);    
#         optimize(M, X, 4; apply_rescaling = true)
    end
    
    @btime begin
        Nx = 20
        Ne = 300
        m = 10
        X = randn(Nx, Ne) .* randn(Nx, Ne) .+ rand(Nx);
        X0 = deepcopy(X);   
        M = HermiteMap(m, X);    
        optimize(M, X, 6; apply_rescaling = true)
    end
end

timing (generic function with 1 method)

In [ ]:
timing()

### Greedyfit

In [53]:
Nx = 10
Ne = 300
m = 20

# X = Matrix([0.267333   1.43021;
#       0.364979   0.607224;
#      -1.23693    0.249277;
#      -2.0526     0.915629;
#      -0.182465   0.415874;
#       0.412907   1.01672;
#       1.41332   -0.918205;
#       0.766647  -1.00445]');

X = randn(Nx, Ne) .* randn(Nx, Ne) .+ rand(Nx)
L = LinearTransform(X; diag = true)

transform!(L, X)

10×300 Array{Float64,2}:
  0.915567    -0.796065   -0.731721   …   0.529433   -0.303699   -1.14415
 -0.00186175   0.333526   -0.463683      -0.150861    0.233496    0.104
 -0.00882651  -0.628545    0.0304819      5.11847     1.43685     0.19322
  0.879108     0.196255    0.674179      -0.107603   -0.0346852   1.62582
 -1.3971      -0.0416449   0.0445271      0.10607    -0.320214    1.35214
  0.496067    -0.0639364  -0.965482   …   0.105975    0.0746329   2.6746
  0.344445     0.105641    0.853668      -0.598025   -0.0165297   0.111082
  0.19992     -0.0412068  -1.38972       -0.127004   -0.04023     0.147524
 -0.847076     0.102524   -0.220191       0.0462358  -0.171279    3.46266
  0.135713    -1.64341     1.28061       -1.47047     0.290523   -0.0167174

In [59]:
@time greedyfit(m, Nx, X, 8; verbose = false);

  0.025685 seconds (31.35 k allocations: 42.543 MiB, 17.06% gc time)


In [ ]:
@time greedyfit($m, $Nx, $X, $8; verbose = false);